In [2]:
import pandas as pd

In [3]:
df = pd.read_pickle('../data/processed/sustainalytics_dataset.pkl')
df.head()

,name,rating,content
0,1&1 Drillisch AG,21.1,1&1 Drillisch AG (formerly known as: Drillisch...
1,2i Rete Gas SpA,37.2,"Headquartered in Milan, 2i Rete Gas is the se..."
2,"2U, Inc.",15.7,"2U, Inc. (formerly 2tor Inc.) is an American e..."
4,3i Group PLC,12.6,3i Group plc is a British multinational privat...
5,3i Infrastructure PLC,22.1,3i Infrastructure plc (LSE: 3IN) is an investm...


In [4]:
import re

def clean_text(content):
    content = re.sub('\n', ' ', content)
    content = re.sub('[=]{2,}', ' ', content)
    content = re.sub('[ ]{2,}', ' ', content)
    return content

df['content'] = df['content'].apply(clean_text)

In [5]:
X = df['content']
y = df['rating']

len(X)

3064

In [6]:
print(min(y))
print(max(y))
print(sum(y)/len(y))

6.2
67.4
25.239915143603177


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print(len(X_train))
print(len(X_test))

2144
920


In [8]:
# Nombre moyens de mots par article wiki
df['content'].apply(lambda content: len(content.split(' '))).mean()

1443.3697780678851

In [1]:
from transformers import RobertaConfig, RobertaModel
from transformers import RobertaTokenizer

configuration = RobertaConfig()
model = RobertaModel(configuration)
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

Downloading: 100%|██████████| 899k/899k [00:01<00:00, 760kB/s]
Downloading: 100%|██████████| 456k/456k [00:00<00:00, 637kB/s]


In [10]:
tokenizer("Hello guys & girls")

{'input_ids': [0, 31414, 1669, 359, 1972, 2], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [23]:
from math import ceil

In [24]:
# max_length = 200
max_length = 10
# overlap_length = 50
overlap_length = 5

def split_tokens_sequence(tokens_sequence):
    n = len(tokens_sequence)
    sequences = list()
    sequences.append(tokens_sequence[0 : min(max_length, n)])
    
    for i in range(ceil((n - max_length) / (max_length - overlap_length))):
        initial_pos = (max_length - overlap_length)*(i + 1)
        final_pos = min((max_length - overlap_length)*(i + 1) + max_length, n)
        sequences.append(tokens_sequence[initial_pos : final_pos])
    
    return sequences